# Black-Scholes solution
The value of a European call option is:

$$
C(S_t,t) = N(d_1)S_t - N(d_2)PV(K) \\
d_1 = \frac{1}{\sigma\sqrt{T-t}} \\
d_2 = d_1-\sigma\sqrt{T-t} \\
PV(K) = Ke^{r(T-t)}
$$

Under the NA0 (No Arbitrage Opportunity) the assumption the **put-call parity** exists as:

$$
C(S_t,t) - P(S_t,t) = S_t - PV(K) \\
PV(K) = Ke^{r(T-t)}
$$

The value of a European put option is:

$$
P(S_t,t) =  N(-d_2)PV(K) - N(-d_1)S_t
$$

$N(.)$ is the cumulative distribution function of the standard normal distribution <br>
$K$ is the strike price <br>
$T-t$ is the time to maturity (expressed in years)<br>
$S_t$ is the spot price of the underlying asset<br>
$r$ is the risk-free rate (annual rate, expressed in terms of continuous compounding)<br>
$\sigma$ is the implied volatility of returns of the underlying asset.<br>


In [1]:
from typing import Literal
import numpy as np
import scipy.stats as stats

from modules.pricings import (
    black_scholes_option_price,
    from_call_to_put,
    from_put_to_call,
)


In [2]:
call_price = black_scholes_option_price(100, 110, 0.05, 1, 0.2, "call")

print(f"Call price: $ {call_price:.2f}")

put_price = black_scholes_option_price(100, 110, 0.05, 1, 0.2, "put")
print(f"Put price using the BS formula: $ {put_price:.2f}")
print(
    f"Put price using the put call parity formula: $ {from_call_to_put(call_price,100,110,0.05,1):.2f}"
)


Call price: $ 6.04
Put price using the BS formula: $ 10.68
Put price using the put call parity formula: $ 10.68


# Monte Carlo simulation 

Given the current asset price at time $0$ is $S_0$, then the asset price at time $T$ can be expressed as:

$$
S_T = S_0e^{(r- \frac{\sigma^2}{2})T  + \sigma W_T}
$$

$T$ is the time to maturity<br>
$S_t$ is the spot price of the underlying asset<br>
$r$ is the risk-free rate (annual rate, expressed in terms of continuous compounding)<br>
$\sigma$ is the volatility of returns of the underlying asset.<br>
$W_T$ is the Brownian motion.<br>

Where $W_T$ follows the normal distribution with mean $0$ and variance $T$. The pay-off of the call option is $max(S_T−K,0)$ and for the put option is $max(K−ST)$.


In [4]:
from modules.pricings import monte_carlo_option_price

In [5]:
call_price, call_price_std_error = monte_carlo_option_price(
    s0=100, k=110, r=0.05, tau=1, sigma=0.2, nb_simulation=100000, option_type="call"
)
print(f"Call price: $ {call_price:.2f} +/- {call_price_std_error:.2f}")
put_price, put_price_std_error = monte_carlo_option_price(
    s0=100, k=110, r=0.05, tau=1, sigma=0.2, nb_simulation=100000, option_type="put"
)
print(f"Put price: $ {put_price:.2f} +/- {put_price:.2f}")


Call price: $ 6.03 +/- 0.04
Put price: $ 6.05 +/- 6.05


# Comparing Black-Scholes formula to Monte-Carlo simulation
## Call options


In [6]:
bs_call_price = black_scholes_option_price(100, 110, 0.05, 1, 0.2, "call")
mc_call_price, mc_call_price_std = monte_carlo_option_price(
    100, 110, 0.05, 1, 0.2, 1000, "call"
)

print(f"Black-Scholes call price: $ {bs_call_price:.2f}")
print(f"Monte-Carlo call price: $ {mc_call_price:.2f} +/- {mc_call_price_std:.2f}")


Black-Scholes call price: $ 6.04
Monte-Carlo call price: $ 5.72 +/- 0.35


## Put options


In [7]:
bs_put_price = black_scholes_option_price(100, 110, 0.05, 1, 0.2, "put")
mc_put_price, mc_put_price_std = monte_carlo_option_price(
    100, 110, 0.05, 1, 0.2, 100000, "put"
)

print(f"Black-Scholes put price: $ {bs_put_price:.2f}")
print(f"Monte-Carlo put price: $ {mc_put_price:.2f} +/- {mc_put_price_std:.2f}")


Black-Scholes put price: $ 10.68
Monte-Carlo put price: $ 5.96 +/- 0.04
